<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PTB文本数据集介绍" data-toc-modified-id="PTB文本数据集介绍-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PTB文本数据集介绍</a></span></li><li><span><a href="#PTB数据集的操作" data-toc-modified-id="PTB数据集的操作-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>PTB数据集的操作</a></span><ul class="toc-item"><li><span><a href="#ptb_raw_data" data-toc-modified-id="ptb_raw_data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>ptb_raw_data</a></span></li><li><span><a href="#ptb_producer" data-toc-modified-id="ptb_producer-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>ptb_producer</a></span></li></ul></li><li><span><a href="#参看资料" data-toc-modified-id="参看资料-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>参看资料</a></span></li></ul></div>

# PTB文本数据集介绍

PTB(Penn Treebank Dataset)文本数据集是语言模型学习中目前最广泛使用的数据集。

**PTB下载地址**

http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-example.tgz

PTB下会有很多文件夹列表, 但是我们只关心data数据集下下列三个文件:

- ptb.test.txt:测试集数据文件
- ptb.train.txt:训练集数据文件
- ptb.valid.txt:验证集数据文件

这三个数据文件的数据已经进过了预处理, 包含了 $10000$ 个不同的词语和语句结束标记符(在文本中就是换行符)以及标记稀有词语的特殊符号 `<unk>`. 下面展示了训练数据中的一行:

`mr. <unk> is chairman of <unk> n.v. the dutch publishing group`

# PTB数据集的操作

TensorFlow 提供了两个函数来帮助实现数据的预处理.

- ptb_raw_data: 读取PTB的原始数据, 并将原始数据中的单词转化为单词ID;
- ptb_producer: 实现截断, 并把数据组成成batch;

## ptb_raw_data

In [3]:
import tensorflow as tf
import reader

In [5]:
# 数据路径
DATA_PATH = "../../../TensorFlow/datasets/PTB_data"

# 读取数据
train_data, valid_data, test_data, _ = reader.ptb_raw_data(DATA_PATH)

# 读取原始数据
print("data len: ", len(train_data))
print("data[0:20]: ", train_data[:30])

data len:  929589
data[0:20]:  [9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 2, 9256, 1, 3, 72, 393]


从上述数据可知, 该训练集中总共包含 $929589$ 个单词, 对比原句可知, 句子的结束ID为 $2$.

## ptb_producer

虽然循环神经网路可以接受任意长度的序列, 但是在训练时需要将序列按照某个固定的长度来阶段, 并将数据组织成batch; ptb_producer函数会将一个长序列划分为 batch_size 段, 其中 batch_size 为一个 batch 的大小, batch 中每一条数据的长度为 num_step.

![ptb_batch](ptb_batch.png)

每次需要往模型中输入的值就是图中虚线框的值.

In [8]:
# 将训练数据组织成batch大小为4, 截断长度为5的数据组
# ptb_producer返回一个二维的tuple数据
result = reader.ptb_producer(train_data, 4, 5)

# 通过队列依次读取batch
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 生成3组batch
    for i in range(3):
        x, y = sess.run(result)
        print("X%d: \n"% i, x)
        print("Y%d: \n"% i, y)
        print("\n")
        
    coord.request_stop()
    coord.join(threads)

X0: 
 [[9970 9971 9972 9974 9975]
 [ 332 7147  328 1452 8595]
 [1969    0   98   89 2254]
 [   3    3    2   14   24]]
Y0: 
 [[9971 9972 9974 9975 9976]
 [7147  328 1452 8595   59]
 [   0   98   89 2254    0]
 [   3    2   14   24  198]]


X1: 
 [[9976 9980 9981 9982 9983]
 [  59 1569  105 2231    1]
 [   0  312 1641    4 1063]
 [ 198  150 2262   10    0]]
Y1: 
 [[9980 9981 9982 9983 9984]
 [1569  105 2231    1  895]
 [ 312 1641    4 1063    8]
 [ 150 2262   10    0  507]]


X2: 
 [[9984 9986 9987 9988 9989]
 [ 895    1 5574    4  618]
 [   8  713    0  264  820]
 [ 507   74 2619    0    1]]
Y2: 
 [[9986 9987 9988 9989 9991]
 [   1 5574    4  618    2]
 [ 713    0  264  820    2]
 [  74 2619    0    1    8]]




从上面的结果可以看出, 在第一个batch 的第一行中, 前面5个单词的ID和整个训练数据中前5个单词的ID是对应的. ptb_producer会自动生成每个batch对应的正确答案, 这对于每个单词, 它对应的正确答案就是该单词的后面的一个单词.

# 参看资料

1. 《TensorFlow实战google深度学习框架》